In [4]:
import time
import requests
import pandas as pd

API_KEY = "XXXX"  # <-- put your key here
SYMBOLS = ["AAPL", "GOOGL", "NVDA"]
SLEEP_SECONDS = 15  # free tier safety

def fetch_earnings(symbol: str) -> dict:
    url = "https://www.alphavantage.co/query"
    params = {"function": "EARNINGS", "symbol": symbol, "apikey": API_KEY}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

all_rows = []

for symbol in SYMBOLS:
    print(f"Fetching EPS for {symbol}...")
    data = fetch_earnings(symbol)

    if "quarterlyEarnings" not in data:
        print("  Unexpected response:", data)
        time.sleep(SLEEP_SECONDS)
        continue

    df = pd.DataFrame(data["quarterlyEarnings"])
    df["symbol"] = symbol
    all_rows.append(df)

    time.sleep(SLEEP_SECONDS)

eps_df = pd.concat(all_rows, ignore_index=True)

# Clean up types
eps_df["fiscalDateEnding"] = pd.to_datetime(eps_df["fiscalDateEnding"], errors="coerce")
for c in ["reportedEPS", "estimatedEPS", "surprise", "surprisePercentage"]:
    if c in eps_df.columns:
        eps_df[c] = pd.to_numeric(eps_df[c], errors="coerce")

eps_df = eps_df.sort_values(["symbol", "fiscalDateEnding"], ascending=[True, False])

eps_df.to_csv(
    r"C:\Users\adyat\OneDrive\Desktop\Adrian\LSE Data Analytics Career Accelerator\Employer Project\av_eps_quarterly.csv",
    index=False
)
print("Saved: av_eps_quarterly.csv")

eps_df.head()

Fetching EPS for AAPL...
Fetching EPS for GOOGL...
Fetching EPS for NVDA...
Saved: av_eps_quarterly.csv


,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,symbol
0,2025-09-30,2025-10-30,1.85,1.76,0.09,5.1136,post-market,AAPL
1,2025-06-30,2025-07-31,1.57,1.43,0.14,9.7902,post-market,AAPL
2,2025-03-31,2025-05-01,1.65,1.62,0.03,1.8519,post-market,AAPL
3,2024-12-31,2025-01-30,2.40,2.34,0.06,2.5641,post-market,AAPL
4,2024-09-30,2024-10-31,0.97,0.95,0.02,2.1053,post-market,AAPL
